# Query the Senator Knowledge Graph

This notebook provides the code for various querying of the senator knowledge graph built in build_graph.ipynb

In [ ]:
import pandas as pd
import neo4j
import seaborn as sns
pd.options.display.max_colwidth = 150

In [ ]:
driver = neo4j.GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "senate"))

In [ ]:

def run(query):
    data = []
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            data.append(record.data())
    return pd.DataFrame(data)

# Example Queries

## --- Simple Queries ---

### (1) Get all bills Bernie Sanders voted yea on

In [ ]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[v:Voted {vote_type: "yea"}]-(r:Rollcall)
RETURN v.vote_type, r.desc, r.bill_number
""")

### (2) Get all of Bernie Sander tweets ordered by likes

In [ ]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[tw:Tweeted]->(t:Tweet)
RETURN t.text, t.likes
ORDER BY t.likes DESC
""")

### (3) Get all of Bernie Sanders' top donors

In [ ]:
run("""
MATCH (c:Contributor)-[d:DonatedTo]->(s:Senator {name: "Bernie Sanders"})
RETURN c.name, d.total
ORDER BY d.total DESC
""")

### (4) Get the amount each senator received in donations from individuals and from pacs (+ plot)

In [ ]:
senator_donation_amounts = run("""
MATCH (c:Contributor)-[d:DonatedTo]->(s:Senator)
RETURN s.name as name, s.party as party, sum(d.pac) as pac_donations, sum(d.individual) as individual_donations
ORDER BY sum(d.individual) DESC
""")
print(senator_donation_amounts)


sns.scatterplot(data=senator_donation_amounts, y="individual_donations", x="pac_donations", hue="party", 
    palette=dict(republican="red", democrat="blue", other="orange")
)
sns.set(rc={'figure.figsize':(16, 9)})

## --- Intermediate Level Queries ---

### (5) Get the organization entities mentioned in roll calls for which Bernie Sander voted yea on, and order by organization count 

In [ ]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[v:Voted {vote_type: "yea"}]->(r:Rollcall)-[m:Mentions]->(e:Entity {type: "ORG"})
RETURN e.name, count(e)
ORDER BY count(e) DESC
""")

### (6) Get the organization entities that Bernie Sanders tweets about, and order by organization count

In [ ]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[tw:Tweeted]->(t:Tweet)-[m:Mentions]->(e:Entity {type: "ORG"})
RETURN e.name, count(e)
ORDER BY count(e) DESC
""")

### (7) Get the event entities that Republicans tweet about, and order by the event count

In [ ]:
run("""
MATCH (s:Senator {party: "republican"})-[tw:Tweeted]->(t:Tweet)-[m:Mentions]->(e:Entity {type: "EVENT"})
RETURN e.name, count(e)
ORDER BY count(e) DESC
""")

### (8) Get total contributions and total tweet likes per senator (+ plot)

In [ ]:
senator_donation_likes = run("""
MATCH (c:Contributor)-[d:DonatedTo]->(s:Senator)-[tw:Tweeted]->(t:Tweet)
RETURN s.name as name, sum(d.total) as donation_totals, s.party as party, sum(t.likes) as tweet_likes
ORDER BY sum(t.likes) DESC
""")
print(senator_donation_likes)
sns.scatterplot(
    data=senator_donation_likes, x="tweet_likes", y="donation_totals", hue="party", 
    palette=dict(republican="red", democrat="blue", other="orange")
)
sns.set(rc={'figure.figsize':(20, 9)})

### (9) Get the senators that have been tweeting about covid the most

In [ ]:
run("""
MATCH (s:Senator)-->(t:Tweet)
WHERE toLower(t.text) CONTAINS "covid"
RETURN s.name, s.party, count(s)
ORDER BY count(s) desc
""")

## --- More Advanced Queries ---

### (10) Get the contributers who supported Senators that voted yes on bills mentioning 'the environmental protection agency'. Return the contributor, senator, and the amount.

In [ ]:
run("""
MATCH (s:Senator)-[v:Voted {vote_type: "yea"}]->(r:Rollcall)-[m:Mentions]->(e:Entity {name: "the environmental protection agency"})
WITH s
MATCH (c:Contributor)-[d:DonatedTo]->(s)
RETURN c.name as contributor, sum(d.total) as amount, s.name as senator
ORDER BY sum(d.total) DESC
""")

### (11) Find the percentage of donations for each candidate that come from universities

In [ ]:
run("""
MATCH (c:Contributor)-[d:DonatedTo]->(s:Senator)
WHERE toLower(c.name) CONTAINS "university"
WITH s, sum(d.total) as university_donations

MATCH (c:Contributor)-[d:DonatedTo]->(s)
RETURN s.name, s.party, s.state, toFloat(university_donations) / toFloat(sum(d.total)) as university_percentage

ORDER BY university_percentage DESC
""")

### (12) Of the above senators, have any tweeted about student loans? 

In [ ]:
run("""
MATCH (c:Contributor)-[d:DonatedTo]->(s:Senator)
WHERE toLower(c.name) CONTAINS "university"
WITH s, sum(d.total) as university_donations

MATCH (c:Contributor)-[d:DonatedTo]->(s)
WITH s, toFloat(university_donations) / toFloat(sum(d.total)) as university_percentage

MATCH (s:Senator)-[tw:Tweeted]->(t:Tweet)
WHERE toLower(t.text) CONTAINS "student loan"
RETURN s.name, university_percentage, t.text, t.likes

ORDER BY university_percentage DESC
""")

### (13) Of democratic senators who frequently cross the aisle, who are their most popular donors?
### * crossing the aisle defined as voting on a bill the majority of republicans voted on and fewer than 5 other democrats voted on

In [ ]:
run("""
MATCH (rs:Senator {party:"republican"})-[rv:Voted {vote_type: "yea"}]->(r:Rollcall)
with r, count(rv) as republican_votes
MATCH (ds:Senator {party:"democrat"})-[dv:Voted {vote_type: "yea"}]->(r:Rollcall)
with r, count(dv) as democrat_votes, republican_votes
WHERE republican_votes > 23 and democrat_votes < 5

MATCH (ds:Senator {party:"democrat"})-[dv:Voted {vote_type: "yea"}]->(r:Rollcall)
WITH ds as democrat_aisle_crosser, count(ds) as dissenting_votes
WHERE count(ds) > 10

MATCH (c:Contributor)-[d:DonatedTo]->(democrat_aisle_crosser)
RETURN c.name, sum(d.total)
ORDER BY sum(d.total) DESC
""")

### Compare that to the top contributors among all democrats

In [ ]:
run("""
MATCH (c:Contributor)-[d:DonatedTo]->(ds:Senator {party:"democrat"})
RETURN c.name, sum(d.total)
ORDER BY sum(d.total) DESC
""")

### (14) Of all senators who received donations from contributors with "oil" in the name, how did they vote on approving the keystone xl pipeline?

In [ ]:
run("""
match (c:Contributor)-[d:DonatedTo]->(s:Senator)-[v:Voted]->(r:Rollcall)    
where toLower(c.name) contains "oil" and toLower(r.desc) contains "keystone xl"
return s.name, sum(d.total) as oil_donations, v.vote_type
order by sum(d.total) desc
""")

### (15) Of the above senators, do any have tweets mentioning oil?

In [ ]:
run("""
match (c:Contributor)-[d:DonatedTo]->(s:Senator)-->(t:Tweet)
match (s)-[v:Voted]->(r:Rollcall)    
where toLower(c.name) contains "oil" and toLower(t.text) contains " oil" and toLower(r.desc) contains "keystone xl" 
return s.name, sum(d.total) as oil_donations, v.vote_type, t.text
order by sum(d.total) desc
""")